# Defect Injection in SPICE Netlists
This Jupyter Notebook was created to explore injecting defects into SPICE netlists based in the IEEE P2427 framework.

It involves the following process:

1.   Devices and nodes are in the netlist are parsed.
2.   Defects are added into the netlist. These are modelled as large parameter changes, shorts or opens.
3.   The netlist is simulated and performance parameters are measured. If the performance parameters fall outside of the expected range, they are classified as failures.
4. Defect coverage is determined based in the percentage of defects detected by the tests.

In [46]:
# Import modules
import re

## Example SPICE Netlist

This netlist is a testbench for testing a simple 5 transistor OpAmp.

In [47]:
# Download SPICE netlist from github
spice_netlist_url = f"https://raw.githubusercontent.com/TimothyJNewman/open-source-chip-design-v1/refs/heads/main/ISSCC_2025_V1/netlist/test_analog.spice"
!wget {spice_netlist_url}
!mkdir generated_netlists

--2024-11-25 23:18:00--  https://raw.githubusercontent.com/TimothyJNewman/open-source-chip-design-v1/refs/heads/main/ISSCC_2025_V1/netlist/test_analog.spice
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5344 (5.2K) [text/plain]
Saving to: ‘test_analog.spice’

test_analog.spice   100%[===================>]   5.22K  --.-KB/s    in 0s      

2024-11-25 23:18:00 (49.9 MB/s) - ‘test_analog.spice’ saved [5344/5344]

mkdir: cannot create directory ‘generated_netlists’: File exists


In [48]:
# Open SPICE netlist
with open("test_analog.spice", "r") as netlist_file:
  netlist_contents = netlist_file.read()

# Print file in terminal
  print(netlist_contents)

** sch_path: /home/timothyjabez/Documents/Open_Source_Circuit_Design/ISSCC_2025_V1/xschem/Benchmark_Circuits/test_analog/test_analog.sch
**.subckt test_analog
V6 vin_p GND 0.9 dc 0.9 ac 1 pulse(0 1 100p 100p 100p 1u 2u)
V7 vin_n GND 0.9
V8 VDD GND 1.8
V9 VSS GND 0
I0 VSS i_bias 10u
x1 VDD vout vin_p vin_n VSS i_bias en OpAmp
V1 en GND 1.8
C1 vout GND 1p m=1
**** begin user architecture code
.lib /usr/local/share/pdk/sky130A/libs.tech/combined/sky130.lib.spice tt
**** end user architecture code
**.ends

* expanding   symbol:  Benchmark_Circuits/OpAmp_Defect/OpAmp.sym # of pins=7
** sym_path: /home/timothyjabez/Documents/Open_Source_Circuit_Design/ISSCC_2025_V1/xschem/Benchmark_Circuits/OpAmp_Defect/OpAmp.sym
** sch_path: /home/timothyjabez/Documents/Open_Source_Circuit_Design/ISSCC_2025_V1/xschem/Benchmark_Circuits/OpAmp_Defect/OpAmp.sch
.subckt OpAmp VDD vout vin_p vin_n VSS i_bias en
*.ipin vin_p
*.ipin vin_n
*.iopin VSS
*.iopin VDD
*.opin vout
*.ipin i_bias
*.ipin en
XM1 vout Active_

## Parsing FET devices
```
Fet structure (both n and p):
<inst_name> <drain> <gate> <source> <bulk> <cell_name>

Example:
XM1 vout Active_load_g VDD VDD sky130_fd_pr__pfet_01v8 ...
```



In [49]:
def get_fet_devices(netlist_contents):

  # Regular expression to extract FET devices (lines starting with XM)
  regex = r'(?P<inst_name>\w+\d+)\s(?P<drain>\w+)\s(?P<gate>\w+)\s(?P<source>\w+)\s(?P<bulk>\w+)\s(?P<cell_name>sky130_fd_pr__[np]fet_01v8)'

  # Find all matching FET device lines
  fet_devices = re.findall(regex, netlist_contents)

  # Display the extracted FET devices
  #for fet in fet_devices:
  #    print("Inst_name", fet[0])
  #    print("Drain", fet[1])
  #    print("Gate", fet[2])
  #    print("Source", fet[3])
  #    print("Bulk", fet[4])
  #    print("Cell_name", fet[5])
  #    print("")
  return fet_devices

defect_terminal_dict = {
  "Drain": 1,
  "Gate": 2,
  "Source": 3,
  "Bulk": 4,
}

## Add Open Defect to SPICE netlist


In [50]:
def modify_nelist_open_defect(netlist, fet_device_terminals, defect_device, defect_terminal_name):

  # Get terminal number
  defect_terminal_num = defect_terminal_dict[defect_terminal_name]
  defect_terminal_netname = fet_device_terminals[defect_terminal_num]

  # Create 2 additional nets to replace original connected to device terminal
  defect_terminal_netname_replaced_1 = "{}1".format(defect_terminal_netname)
  defect_terminal_netname_replaced_2 = "{}2".format(defect_terminal_netname)
  defect_terminal_netname_replaced_3 = "{}3".format(defect_terminal_netname)

  fet_device_terminals_copy = fet_device_terminals.copy()
  # String of original device terminals
  original_device_terminals = " ".join(fet_device_terminals_copy)

  fet_device_terminals_copy[defect_terminal_num] = defect_terminal_netname_replaced_1

  # String of modified device terminals
  modified_device_terminals = " ".join(fet_device_terminals_copy)

  print("Open at", defect_terminal_name, fet_device_terminals_copy)

  # Replace device terminals
  modified_netlist = re.sub(original_device_terminals, modified_device_terminals, netlist)
  # Replace remain occurances of net
  modified_netlist = re.sub(r"\b{}\b".format(defect_terminal_netname), defect_terminal_netname_replaced_2, modified_netlist)

  if (defect_terminal_name == "Drain"):
    # Model open defect by adding a resistor
    modified_netlist = re.sub("[\*]{2}.ends", "R_drain_open {} {} 1G m=1 \n**.ends"
      .format(defect_terminal_netname_replaced_1, defect_terminal_netname_replaced_2),
                              modified_netlist, count=1)
  elif (defect_terminal_name == "Source"):
    # Model open defect by adding a resistor
    modified_netlist= re.sub("[\*]{2}.ends", "R_source_open {} {} 1G m=1 \n**.ends"
      .format(defect_terminal_netname_replaced_1, defect_terminal_netname_replaced_2),
                              modified_netlist, count=1)
  elif (defect_terminal_name == "Gate"):
    # Model open defect for gate by adding a resistor network
    modified_netlist = re.sub("[\*]{2}.ends", "R_gate1_drain_open {gate1} {drain} 0.5T m=1 \nR_gate1_source_open {gate1} {source} 0.5T m=1 \nR_gate1_gate_open {gate1} {gate} 1T m=1 \nR_gate1_gate2_open {gate1} {gate2} 100T m=1 \n**.ends"
      .format(gate=defect_terminal_netname_replaced_1, gate2=defect_terminal_netname_replaced_2, gate1=defect_terminal_netname_replaced_3, drain=fet_device_terminals_copy[1], source=fet_device_terminals_copy[3]),
                              modified_netlist, count=1)

  return modified_netlist

def modify_nelist_short_defect(netlist, fet_device_terminals, defect_device, defect_terminal_name_1, defect_terminal_name_2):

  print("Short at", defect_terminal_name_1, "-", defect_terminal_name_2, fet_device_terminals)

  # Get terminal number
  defect_terminal_num_1 = defect_terminal_dict[defect_terminal_name_1]
  defect_terminal_netname_1 = fet_device_terminals[defect_terminal_num_1]
  defect_terminal_num_2 = defect_terminal_dict[defect_terminal_name_2]
  defect_terminal_netname_2 = fet_device_terminals[defect_terminal_num_2]

  if ((defect_terminal_name_1 == "Drain" and defect_terminal_name_2 == "Source") \
      or (defect_terminal_name_1 == "Source" and defect_terminal_name_2 == "Drain")):
    # Model open defect by adding a resistor
    modified_netlist = re.sub("[\*]{2}.ends", "R_drain_source_short {} {} 10 m=1 \n**.ends"
      .format(defect_terminal_netname_1, defect_terminal_netname_2),
                              netlist, count=1)

  elif ((defect_terminal_name_1 == "Gate" and defect_terminal_name_2 == "Drain") \
        or (defect_terminal_name_1 == "Drain" and defect_terminal_name_2 == "Source")):
    # Model open defect by adding a resistor
    modified_netlist = re.sub("[\*]{2}.ends", "R_gate_drain_short {} {} 10 m=1 \n**.ends"
      .format(defect_terminal_netname_1, defect_terminal_netname_2),
                              netlist, count=1)

  elif ((defect_terminal_name_1 == "Gate" and defect_terminal_name_2 == "Source") \
        or (defect_terminal_name_1 == "Source" and defect_terminal_name_2 == "Gate")):
    # Model open defect by adding a resistor
    modified_netlist = re.sub("[\*]{2}.ends", "R_gate_source_short {} {} 10 m=1 \n**.ends"
      .format(defect_terminal_netname_1, defect_terminal_netname_2),
                              netlist, count=1)

  return modified_netlist

In [51]:
def generate_spice_netlists(netlist, defect_type, fet_device_terminals, defect_device_num, defect_terminal_name_1, defect_terminal_name_2):

  if (defect_type == "Open"):
    modified_netlist_contents = modify_nelist_open_defect(netlist, fet_device_terminals, defect_device_num, defect_terminal_name_1)
    #print(modified_netlist_contents)
  elif (defect_type == "Short"):
    modified_netlist_contents = modify_nelist_short_defect(netlist, fet_device_terminals, defect_device_num, defect_terminal_name_1, defect_terminal_name_2)

  modified_netlist_contents = re.sub("opamp_specs.txt", "opamp_specs_{}_{}_{}_{}_{}.txt"
      .format(defect_device_num,fet_device_terminals[0],defect_type,defect_terminal_name_1,defect_terminal_name_2),
                              modified_netlist_contents)

  # Open a file for writing modified netlist
  with open("./generated_netlists/test_analog_{}_{}_{}_{}_{}.spice"
    .format(defect_device_num,fet_device_terminals[0],defect_type,defect_terminal_name_1,defect_terminal_name_2), "w") as file:
      file.write(modified_netlist_contents)


In [52]:
# Extract all fet devices from netlist
fet_devices = get_fet_devices(netlist_contents)

# Create netlists with defects injected
for defect_device_num in range(0,len(fet_devices)-1):
  fet_device_terminals = list(fet_devices[defect_device_num])
  generate_spice_netlists(netlist_contents, "Open", fet_device_terminals, defect_device_num, defect_terminal_name_1="Drain", defect_terminal_name_2=None)
  generate_spice_netlists(netlist_contents, "Open", fet_device_terminals, defect_device_num, defect_terminal_name_1="Source", defect_terminal_name_2=None)
  generate_spice_netlists(netlist_contents, "Open", fet_device_terminals, defect_device_num, defect_terminal_name_1="Gate", defect_terminal_name_2=None)
  generate_spice_netlists(netlist_contents, "Short", fet_device_terminals, defect_device_num, defect_terminal_name_1="Drain", defect_terminal_name_2="Source")
  generate_spice_netlists(netlist_contents, "Short", fet_device_terminals, defect_device_num, defect_terminal_name_1="Gate", defect_terminal_name_2="Drain")
  generate_spice_netlists(netlist_contents, "Short", fet_device_terminals, defect_device_num, defect_terminal_name_1="Gate", defect_terminal_name_2="Source")

Open at Drain ['XM1', 'vout1', 'Active_load_g', 'VDD', 'VDD', 'sky130_fd_pr__pfet_01v8']
Open at Source ['XM1', 'vout', 'Active_load_g', 'VDD1', 'VDD', 'sky130_fd_pr__pfet_01v8']
Open at Gate ['XM1', 'vout', 'Active_load_g1', 'VDD', 'VDD', 'sky130_fd_pr__pfet_01v8']
Short at Drain - Source ['XM1', 'vout', 'Active_load_g', 'VDD', 'VDD', 'sky130_fd_pr__pfet_01v8']
Short at Gate - Drain ['XM1', 'vout', 'Active_load_g', 'VDD', 'VDD', 'sky130_fd_pr__pfet_01v8']
Short at Gate - Source ['XM1', 'vout', 'Active_load_g', 'VDD', 'VDD', 'sky130_fd_pr__pfet_01v8']
Open at Drain ['XM2', 'Active_load_g1', 'Active_load_g', 'VDD', 'VDD', 'sky130_fd_pr__pfet_01v8']
Open at Source ['XM2', 'Active_load_g', 'Active_load_g', 'VDD1', 'VDD', 'sky130_fd_pr__pfet_01v8']
Open at Gate ['XM2', 'Active_load_g', 'Active_load_g1', 'VDD', 'VDD', 'sky130_fd_pr__pfet_01v8']
Short at Drain - Source ['XM2', 'Active_load_g', 'Active_load_g', 'VDD', 'VDD', 'sky130_fd_pr__pfet_01v8']
Short at Gate - Drain ['XM2', 'Active_loa

In [55]:
# Code to replace string in all spice files
# This was created to replace .lib path for netlists created locally
# but should be unnecessary when generating all spice files on Jupyter
import os
def replace_string_in_spice_files(folder_path, original_pdk_path, final_pdk_path):

  # Iterate over files in directory
  for name in os.listdir(folder_path):
    # Open file
    with open(os.path.join(folder_path, name), 'r') as file:
      # Read content of file
      netlist = file.read()

    modified_netlist = re.sub(original_pdk_path, final_pdk_path, netlist)

    with open(os.path.join(folder_path, name), 'w') as file:
      file.write(modified_netlist)

replace_string_in_spice_files("/content/generated_netlists", "/usr/local/share/pdk/sky130A/", "/foss/pdks/sky130A/")

In [54]:
# Download generated netlists
!zip -r generated_netlists.zip /content/generated_netlists

from google.colab import files
files.download("/content/generated_netlists.zip")

updating: content/generated_netlists/ (stored 0%)
updating: content/generated_netlists/test_analog_4_XM5_Open_Gate_None.spice (deflated 76%)
updating: content/generated_netlists/test_analog_6_XM9_Open_Drain_None.spice (deflated 76%)
updating: content/generated_netlists/test_analog_5_XM6_Open_Source_None.spice (deflated 77%)
updating: content/generated_netlists/test_analog_2_XM3_Short_Gate_Source.spice (deflated 77%)
updating: content/generated_netlists/test_analog_7_XM10_Short_Gate_Drain.spice (deflated 76%)
updating: content/generated_netlists/test_analog_5_XM6_Open_Gate_None.spice (deflated 76%)
updating: content/generated_netlists/test_analog_2_XM3_Open_Drain_None.spice (deflated 77%)
updating: content/generated_netlists/test_analog_3_XM4_Short_Gate_Source.spice (deflated 76%)
updating: content/generated_netlists/test_analog_4_XM5_Short_Gate_Drain.spice (deflated 76%)
updating: content/generated_netlists/test_analog_2_XM3_Short_Gate_Drain.spice (deflated 76%)
updating: content/gener

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>